In [0]:
# Check the metadata on the video file
import subprocess
import json

# File path
video_file = "/dbfs/FileStore/rachlenn/4_2021_07_06_14_11_37x.mp4"

cmd = [
    "ffprobe",
    "-v", "error",
    "-select_streams", "v:0",
    "-show_entries", "stream=codec_name,width,height,r_frame_rate",
    "-of", "json",
    video_file
]

result = subprocess.run(cmd, capture_output=True, text=True)
info = json.loads(result.stdout)

video = info["streams"][0]
codec = video["codec_name"]
width = video["width"]
height = video["height"]
num, denom = map(int, video["r_frame_rate"].split("/"))
fps = round(num / denom, 2)

print(f"Codec: {codec}")
print(f"Resolution: {width}x{height}")
print(f"FPS: {fps}")



In [0]:
# Import relevant packages
%pip install opencv-python
import cv2
import os

# Set the directory where the video files are located
input_dir = "/dbfs/FileStore/rachlenn" 

# Set the directory where the extracted frames will be saved
output_dir = "/dbfs/FileStore/rachlenn/extracted_frames"
os.makedirs(output_dir, exist_ok=True)

# Supported video file extensions
video_extensions = ('.mp4')

# Function to extract frames from a single video
def extract_frames(video_path, output_folder, frame_interval=1):
    os.makedirs(output_folder, exist_ok=True) # Create the output folder
    cap = cv2.VideoCapture(video_path) # Open the video file

    if not cap.isOpened(): # Check if the video file was opened successfully
        print(f"Error: Could not open video file {video_path}")
        return

    frame_count = 0 # Counter for total frames
    saved_count = 0 # Counter for saved frames

    while True: # Loop through each frame
        ret, frame = cap.read() # Read the next frame
        if not ret: 
            break # Break the loop if there are no more frames

        if frame_count % frame_interval == 0: # Save the frame if it's a multiple of the frame interval
            frame_filename = os.path.join(output_folder, f"frame{frame_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame) # Save the frame as img
            saved_count += 1 

        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from '{os.path.basename(video_path)}'")

# Loop through each file in the input directory
for filename in os.listdir(input_dir):
    if filename.lower().endswith(video_extensions):
        video_path = os.path.join(input_dir, filename)
        video_name = os.path.splitext(filename)[0]
        video_output_folder = os.path.join(output_dir, video_name)

        extract_frames(video_path, video_output_folder, frame_interval=1)




In [0]:
# Display extracted frames

from PIL import Image

frame_dir = "/dbfs/FileStore/rachlenn/to_download/4_2021_07_28_14_02_53x"

# Show only the first 5 frames
for i, filename in enumerate(sorted(os.listdir(frame_dir))):
    if filename.startswith("frame") and filename.endswith(".jpg"):
        if i >= 5:  # Limit to first 5
            break
        frame_path = os.path.join(frame_dir, filename)
        img = Image.open(frame_path)
        display(img)


In [0]:
# Move the extracted frames to the mounted lab zone directory
dbutils.fs.mv("FileStore/rachlenn/extracted_frames", "mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/extracted_frames", True) 

In [0]:
# Path to the directory where your video files are stored
v_dir = "/dbfs/FileStore/rachlenn"

# Loop through and delete all video files
for filename in os.listdir(v_dir):
    if filename.endswith(".mp4"):
        file_path = os.path.join(v_dir, filename)
        os.remove(file_path)
        print(f"🗑 Deleted: {filename}")

In [0]:
# Extract frame rate for videos
video_path = "/dbfs/FileStore/rachlenn/to_download/4_2021_07_28_14_02_53x"
cap = cv2.VideoCapture(video_path) # Open the video file

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frame rate (FPS): {fps}")

cap.release()